In [2]:
import torch
import wandb
import diffusion_pde as dpde
from pathlib import Path
from hydra import initialize, compose
from omegaconf import OmegaConf

In [7]:
data_path = Path("/home/s204790/dynamical-pde-diffusion/data/heat_no_cond_test.hdf5")
data_path.stem

'heat_no_cond_test'

In [8]:
with initialize(config_path="../conf", version_base=None):
    cfg = compose(config_name="train", overrides=[])

wandb_kwargs = OmegaConf.to_container(cfg.wandb, resolve=True)
wandb_kwargs["name"] = f"{data_path.stem}/dataset"
wandb_kwargs

{'entity': 'philiphohwy-danmarks-tekniske-universitet-dtu',
 'project': 'dynamical-pde-diffusion-final',
 'dir': '/home/s204790/logs',
 'name': 'heat_no_cond_test/dataset',
 'mode': 'online'}

In [9]:
with wandb.init(**wandb_kwargs) as run:
    artifact = wandb.Artifact(name="heat_no_cond_test", type="dataset")
    artifact.add_file(str(data_path))
    artifact.metadata = {
        "description": "test dataset for heat equation without conditioning on time or diffusivity",
    }
    run.log_artifact(artifact)

wandb: Currently logged in as: philiphohwy (philiphohwy-danmarks-tekniske-universitet-dtu) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
api = wandb.Api()